# Projet donnée machine learning  
 #### Gabriel Rochon, Louis de Oliveira & Sofiya Ondriash 
 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
%matplotlib inline

#### Lecture des datasets

In [ ]:
current_directory = "c:/Users/gabriel/Desktop/Dauphine/Dauphine L2/Semestre 2/MIDO/Intro machine learning réseaux de neurones/projet ml"
train_file_path = os.path.join(current_directory, 'X_train_G3tdtEn.csv')
test_file_path = os.path.join(current_directory, 'Y_train_2_XPXJDyy.csv')

# On charge les données
df_train = pd.read_csv('X_train_G3tdtEn.csv')
df_test = pd.read_csv('Y_train_2_XPXJDyy.csv')

#### Opérations sur les colonnes et encodage

Nous avons défini une fonction `ajoutrapportprixquantite` qui ajoute une nouvelle colonne au DataFrame d'entraînement. Cette colonne, nommée `RapportPrixQuantite`, est calculée en divisant la somme des colonnes de prix par la somme des colonnes de quantités de produits achetés, afin d'analyser le rapport entre le prix total et la quantité totale de produits achetés pour chaque transaction.

In [ ]:
# Ajout colonne Rapport prix/quantité 
def ajoutrapportprixquantite(df_train):
    prix = ['cash_price' + str(i+1) for i in range(24)]
    qtt = ['Nbr_of_prod_purchas' + str(i+1) for i in range(24)]
    df_train['RapportPrixQuantite'] = df_train[prix].sum(axis=1) / df_train[qtt].sum(axis=1)
    return df_train

Ici, nous avons défini une fonction `encoder` qui encode les colonnes catégorielles spécifiées. Nous avons généré les noms de colonnes pour les items, makes, et models, puis les combinons dans `cols_to_encode`. Ensuite, nous utilisons `pd.get_dummies` pour effectuer le one-hot encoding sur ces colonnes. Enfin, nous convertissons les valeurs booléennes en entiers pour garantir que toutes les colonnes encodées sont au format numérique pour préparer les données catégorielles pour les modèles que nous allons utiliser par la suite.

In [ ]:
def encoder(df_encoded):
    items = ['item' + str(i+1) for i in range(24)]
    makes = ['make' + str(i+1) for i in range(24)]
    models = ['model' + str(i+1) for i in range(24)]
    goods = ['goods_code' + str(i+1) for i in range(24)]
    
    cols_to_encode = items + makes + models

    df_encoded = pd.get_dummies(df_encoded, columns=cols_to_encode)

    # Convert boolean values to integers
    for col in df_encoded.columns:
        if col.split('_')[0] in items + models + makes: 
            df_encoded[col] = df_encoded[col].astype(int)

    return df_encoded

In [ ]:
def ajoutnombrecolonnesdansitems(df_test):
    item_cols = ['item' + str(i+1) for i in range(24)]
    df_test['nombre item'] = df_test[item_cols].count(axis=1)
    
    return df_test

Nous avons défini trois fonctions pour améliorer la qualité de nos données: 

- La fonction `enlevergood_code` supprime les colonnes spécifiées des codes de produits dans le DataFrame encodé parce que nous voulions éliminer les colonnes inutiles ou redondantes avant de procéder à l'application des modèles aux données afin de réduire le nombre de colonnes à traiter pour le modèle.

- La fonction `remplirde0` remplit les valeurs manquantes des colonnes de prix et de quantités par des zéros, garantissant ainsi que les analyses ultérieures ne soient pas faussées par des valeurs manquantes. 

- La fonction `filtrage` identifie et supprime les colonnes ayant un grand nombre de valeurs nulles (zéros), et crée des indicateurs de rareté pour les modèles, items, et marques. Cela réduit la dimensionnalité des données et met en avant les informations pertinentes, optimisant ainsi les performances des modèles de machine learning.

In [ ]:
# On supprime la colloone 'goods_code' 
def enlevergood_code(df_encoded):
    good = ['goods_code' + str(i) for i in range(1, 25)]
    df_encoded = df_encoded.drop(good, axis=1)
    return df_encoded

In [ ]:
# Remplis de 0 les achats non effectué de manière optimisée

def remplirde0_optimized(df_encoded):
    cols = ['cash_price' + str(i+1) for i in range(24)]
    qtt = ['Nbr_of_prod_purchas' + str(i+1) for i in range(24)]
    df_encoded[cols + qtt] = df_encoded[cols + qtt].fillna(0)
    return df_encoded

In [ ]:
def filtragemodif(df_encoded):
    seuil=0.999*len(df_test)
    cols_final=[]
    col_a_supprimer = [col for col in df_encoded.columns if (df_encoded[col] == 0).sum() > seuil]
    for i in range(1, 25):
        model = 'model' + str(i)
        item = 'item' + str(i)
        mark = 'make' + str(i)   
        peupresent = 'peupresent' +str(i)
        cols_to_save_model= [col for col in col_a_supprimer if col.split('_')[0] in model]
        df_encoded['peupresentmodel'+str(i)] = df_encoded[cols_to_save_model].sum(axis=1)
        cols_final.extend(cols_to_save_model)
        cols_to_save_item= [col for col in col_a_supprimer if col.split('_')[0] in item]
        df_encoded['peupresentitem'+str(i)] = df_encoded[cols_to_save_item].sum(axis=1)
        cols_final.extend(cols_to_save_item)
        cols_to_save_mark= [col for col in col_a_supprimer if col.split('_')[0] in mark]
        df_encoded['peupresentmark'+str(i)] = df_encoded[cols_to_save_mark].sum(axis=1)
        cols_final.extend(cols_to_save_mark)
        cols_to_save_peupresent= [col for col in col_a_supprimer if col.split('_')[0] in peupresent]
        cols_final.extend(cols_to_save_peupresent)
    df_encoded = df_encoded.drop(cols_final, axis=1)
    return df_encoded

Ensuite, nous avons essayé une technique pour potentiellement fournir plus d'informations pertinentes au modèle pour affiner la précision.
La fonction `ajoutecartalamoyenne` calcule l'écart par rapport à la moyenne pour chaque item. Elle crée une liste `item` avec les noms des colonnes d'items. Pour chaque colonne, si le nom de base n'est pas dans `item`, elle passe à la suivante. Sinon, elle calcule l'écart entre les valeurs de la colonne et la moyenne des valeurs non nulles, pondérées par le prix en espèces correspondant. Cet écart est normalisé et ajouté à une nouvelle colonne et les valeurs -1 sont remplacées par 0. Théoriquement, cette fonction pourrait aider à quantifier les écarts et d'identifier les anomalies par rapport à la moyenne pour chaque item.

In [ ]:
# Calcule l'écart à la moyenne de chaque catégorie de produit
def calculate_ecart(group):
    moyenne = group.mean()
    ecart = (group - moyenne)/moyenne
    return ecart

# Ajout de la colonne ecart à la moyenne pour chaque item
def ajout_ecart_a_la_moyenne_item(df_test):
    df_test.fillna(0, inplace=True)
    for i in range(1, 25):
        item_col = 'item' + str(i)
        cash_col = 'cash_price' + str(i)
        ecart_col = 'ecartitem' + str(i)
        nb_col = 'Nbr_of_prod_purchas' + str(i)
        df_test[ecart_col] = df_test.groupby(item_col)[cash_col].transform(calculate_ecart)
        df_test[ecart_col] = df_test[ecart_col] * df_test[nb_col]
    df_test.fillna(0, inplace=True)
    return df_test

# Ajout de la colonne ecart à la moyenne pour chaque modèle
def ajout_ecart_a_la_moyenne_modele(df_test):
    df_test.fillna(0, inplace=True)
    for i in range(1, 25):
        model = 'model' + str(i)
        cash_col = 'cash_price' + str(i)
        ecart_col = 'ecartmodel' + str(i)
        nb_col = 'Nbr_of_prod_purchas' + str(i)
        df_test[ecart_col] = df_test.groupby(model)[cash_col].transform(calculate_ecart)
        df_test[ecart_col] = df_test[ecart_col] * df_test[nb_col]
    df_test.fillna(0, inplace=True)
    return df_test

In [ ]:
# Ajout de la colonne prixtotal
def prixtotal(df_test):
    cols = ['cash_price' + str(i) for i in range(1, 25)]
    df_test['prixtotal'] = df_test[cols].sum(axis=1)
    df_test.fillna(0, inplace=True)
    return df_test # ca marche

# Ajout de la colonne qtt
def qtt(df_test):
    cols = ['Nbr_of_prod_purchas' + str(i) for i in range(1, 25)]
    df_test['qtt'] = df_test[cols].sum(axis=1)
    df_test.fillna(0, inplace=True)
    return df_test

# Ajout de la colonne variation de prix
def variationduprix(df_encoded):
    for i in range(1, 24):
        df_encoded['variation'+str(i)] = ((df_encoded['cash_price'+str(i+1)] * df_encoded['Nbr_of_prod_purchas'+str(i+1)])-(df_encoded['cash_price'+str(i)] * df_encoded['Nbr_of_prod_purchas'+str(i)]))
    return df_encoded

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def encodergoodcode (df_train):
    encoder = OrdinalEncoder()
    for i in range(1, 25):
        df_train['goods_code'+str(i)] = df_train['goods_code'+str(i)].astype(str)
        df_train['goods_code'+str(i)] = encoder.fit_transform(df_train['goods_code'+str(i)].values.reshape(-1, 1))
    return df_train

La fonction `tout` applique une série de transformations et de filtrages sur le DataFrame df_test, composée de toutes les fonctions que nous avons détaillées auparavant afin de simplifier et centraliser le filtrage de données.

In [ ]:
def tout(df_test):
    df_test = ajoutnombrecolonnesdansitems(df_test)
    df_test.fillna(0, inplace=True)
    df_test = enlevergood_code(df_test)
    df_test = prixtotal(df_test)
    df_test = qtt(df_test)
    df_test = variationduprix(df_test)
    df_test = ajoutrapportprixquantite(df_test)
    df_test = ajout_ecart_a_la_moyenne_item(df_test)
    df_test = ajout_ecart_a_la_moyenne_modele(df_test)
    df_test = encoder(df_test)
    df_test = remplirde0_optimized (df_test)
    df_test = filtragemodif(df_test)
    return df_test

In [ ]:
df_encoded=tout(df_train)

In [ ]:
# On compare les colonnes après et avant l'encodage 
nb_colonne= df_encoded.shape[1]
print(nb_colonne)
print(df_train.shape[1])

In [ ]:
# On regarde si il a encore des colonnes avec des nan
nan_columns = df_encoded.columns[df_encoded.isna().any()].tolist()
nan_columns

In [ ]:
ID=df_encoded['ID']
X_train=df_encoded
y_train=df_test['fraud_flag']

#### Programmes d'apprentissage et optimisation des programmes

Enfin, nous avons importé la fonction `train_test_split` de sklearn.model_selection, puis divisé les données en ensembles d'entraînement et de validation avec une proportion de 80/20, en utilisant un état aléatoire pour la reproductibilité.

In [ ]:
from sklearn.model_selection import train_test_split

# X_vraitrain et Y_vraitrain sont maintenant nos nouveaux ensembles d'entraînement
# X_val et Y_val sont nos ensembles de validation
X_vraitrain, X_val, y_vraitrain, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=88)

In [ ]:
start = time.time() # On commence le chrono

In [ ]:
# On regarde si il y a des colonnes qui contiennent des valeurs infinies
infinite_columns = df_encoded.columns[(df_encoded == np.inf).any()].tolist()

In [ ]:
infinite_columns # On affiche les colonnes qui contiennent des valeurs infinies

##### RandomForest

Nous utilisons ensuite la fonction `GridSearchCV`afin de trouver les meilleurs paramètres pour entraîner nos modèles. On utilise également GridSearch best estimator pour identifier le modèle optimal à utiliser.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# On utilise GridSerchCV pour trouver les meilleurs hyperparamètres
param_grid = {'n_estimators': [400],'max_depth': [20,25]}

# On utilise 5-fold cross-validation et on parallèlise le processus 
grid_search = GridSearchCV(RandomForestClassifier(random_state=1, n_jobs=-1), param_grid, cv=5) 
grid_search.fit(X_vraitrain, y_vraitrain)

In [ ]:
# On utilise le meilleur modèle trouver par GridSearchCV
model_RF = grid_search.best_estimator_

best_params = grid_search.best_params_

In [ ]:
best_params # On affiche les meilleurs parametres

In [ ]:
model_RF.fit(X_vraitrain, y_vraitrain) # On entraine le modèle

Ici nous avons exécuté plusieurs opérations pour transformer et vérifier la cohérence des ensembles de données d'entraînement et de validation. Ces opérations permettent de s'assurer que les ensembles de validation et d'entraînement ont les mêmes colonnes après transformation, en ajoutant des zéros là où des valeurs sont manquantes. Cela garantit que les modèles de machine learning reçoivent des données cohérentes, facilitant ainsi leur entraînement et leur évaluation. Les différences de colonnes sont également identifiées pour corriger les éventuelles incohérences entre les ensembles de données.

In [ ]:
X_vraitrain.shape # On affiche la taille de notre ensemble d'entraînement

In [ ]:
X_val = X_val.reindex(columns=X_vraitrain.columns) # On réindexe les colonnes de notre ensemble de validation
X_val = X_val.fillna(0) # On remplace les valeurs manquantes par 0
X_val.shape # On affiche la taille de notre ensemble de validation

In [ ]:
# Obtenir les noms des colonnes en tant qu'ensembles
columns_test = set(X_val.columns)
columns_encoded = set(X_vraitrain.columns)

# Trouver les colonnes qui sont dans df_test mais pas dans df_encoded
diff_test_encoded = columns_test - columns_encoded
print("Colonnes dans df_test mais pas dans df_encoded:", diff_test_encoded)

# Trouver les colonnes qui sont dans df_encoded mais pas dans df_test
diff_encoded_test = columns_encoded - columns_test
print("Colonnes dans df_encoded mais pas dans df_test:", diff_encoded_test)


Ici, nous avons évalué les performances du modèle model2 sur l'ensemble de validation X_val en utilisant la métrique d'AUC-PR (Area Under the Precision-Recall Curve) ainsi que la méthode `average precision score` de sklearn.

In [ ]:
from sklearn.metrics import average_precision_score
y_pred_proba = model_RF.predict_proba(X_val)[:, 1]
y_pred = model_RF.predict(X_val)

pr_auc = average_precision_score(y_val, y_pred)
pr_auc2 = average_precision_score(y_val, y_pred_proba)

print(f'PR-AUC: {pr_auc}')
print(f'PR-AUC2: {pr_auc2}')

end = time.time()
print('Le temps est de ', end - start,'secondes')

In [ ]:
X_train.head() # On affiche les 5 premières lignes de notre ensemble d'entraînement

##### KNeighborsClassifier

Nous testons avec le modèle des K-plus proches voisins.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

start=time.time()
nn = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier())
parameters = {'kneighborsclassifier__n_neighbors':[10]}
clf = GridSearchCV(nn, parameters, verbose=1,n_jobs=-1)
clf.fit(X_vraitrain, y_vraitrain)
x=clf.best_params_
print(x)
y_pred = clf.predict_proba(X_val)[:, 1]
pr_auc = average_precision_score(y_val, y_pred)
print(f'PR-AUC: {pr_auc}')
end=time.time()
print('le temps est de ',end-start)

##### Régression logistique 

Ensuite, nous testons aussi le modèle de régression logistique.

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

start=time.time()
nn = make_pipeline(preprocessing.StandardScaler(), LogisticRegression())
parameters = {'logisticregression__C':np.linspace(1, 1000, 10)}
clf2 = GridSearchCV(nn, parameters, verbose=1,n_jobs=-1)
clf2.fit(X_vraitrain, y_vraitrain)
end=time.time()
x=clf2.best_params_
print(x)
z=clf2.best_score_
print(z)
y_pred = clf2.predict_proba(X_val)[:, 1]
pr_auc = average_precision_score(y_val, y_pred)
print(f'PR-AUC: {pr_auc}')
temps=end-start
print("le temps est de ",temps)

#### Importance des variables 

Nous avons également calculé et affiché la matrice de confusion pour les prédictions du modèle, ainsi que les taux de faux positifs (FP) et de faux négatifs (FN).
Idéalement, un bon modèle de détection de fraude doit avoir un faible taux de faux négatifs, car il est crucial de détecter toutes les transactions frauduleuses possibles. Cependant, il est également important de maintenir un taux de faux positifs suffisamment bas pour éviter de perturber les transactions légitimes. L'équilibre entre ces deux taux dépend du coût associé aux faux positifs et aux faux négatifs, et on peut dire qu'ici la métrique plus importante est le faux négatif, car il faut correctement identifier le plus de transactions frauduleuses que possible.

In [ ]:
nouveau = pd.DataFrame()
nouveau['fraud_flag'] = df_test['fraud_flag']
nouveau['ID'] = df_test['ID']
# Sorting the DataFrame by the 'fraud_flag' column
print(nouveau.head())

id = nouveau.loc[nouveau['fraud_flag'] == 1, 'ID'].tolist()
id.sort()   
dico={}
somme=0
s=0
for i in range(1,len(id)):
    for j in range(len(id)):
        if id[j]+i in id:
            somme+=1
    dico[i]=somme
    s+=somme
    somme=0
print(s)
print('Dictionnaire du nombre de fraude en fonction de la distance entre elles')
print(dico)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, ax = plt.subplots()

# Plot the data
ax.plot(list(dico.keys()), list(dico.values()))

# Set the labels and title
ax.set_xlabel('Distance')
ax.set_ylabel('Nombre de Fraudes')
ax.set_title('Nombre de Fraudes en fonction de la distance entre elles')

# Display the plot
plt.show()


In [ ]:
# Convert the dictionary items into a list and take the first 10 items
first_10_items = list(dico.items())[:10]

# Convert the first 10 items back into a dictionary
first_10_dict = dict(first_10_items)

# Now you can plot the first 10 items
fig, ax = plt.subplots()
ax.plot(list(first_10_dict.keys()), list(first_10_dict.values()))
ax.set_xlabel('Distance')
ax.set_ylabel('Nombre de Fraudes')
ax.set_title('Nombre de Fraudes en fonction de la distance entre elles')
plt.show()


In [ ]:
from sklearn.feature_selection import SelectFromModel

# On calcule les importances des variables
importances = model_RF.feature_importances_

# Afficher les 5 variables les plus importantes
indices = sorted(range(len(importances)), key=lambda k: importances[k], reverse=True)[20:]
nom_colonnes = X_train.columns
for i in indices[:5]:
    print(f"{nom_colonnes[i]} : {importances[i]}")

In [ ]:
importances = model_RF.feature_importances_

# Afficher les 5 variables les plus importantes
indices = sorted(range(len(importances)), key=lambda k: importances[k], reverse=True)[:20]
nom_colonnes = X_train.columns
for i in indices:
    print(f"{nom_colonnes[i]} : {importances[i]}")

In [ ]:
# Afficher les 10 variables les plus importantes
nom_colonnes = X_train.columns

# Créer un graphique à barres horizontal pour l'importance des variables
plt.figure(figsize=(12, 8))
plt.barh(range(len(indices)),[importances[i] for i in indices],  align='center')
plt.yticks(range(len(indices)),[nom_colonnes[i] for i in indices])
plt.xlabel('Importance')
plt.ylabel('Variables')
plt.title('Top 20 Variables Importance')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix

# Calculez la matrice de confusion
cm = confusion_matrix(y_val, y_pred)

# Calculez le taux de faux positifs et de faux négatifs
fp_rate = cm[0, 1] / cm[0].sum()
fn_rate = cm[1, 0] / cm[1].sum()

print(f"Taux de faux positif: {fp_rate}")
print(f"Taux de faux négatif: {fn_rate}")

In [ ]:
from sklearn.metrics import classification_report

classification_report(y_val, y_pred) # On affiche le rapport de classification

On a un modele prudent qui dit plus facilement négatif que positif comme on dit vaut mieux 2 criminels dehors qu'un innocent en prison

In [ ]:
model_RFF = model_RF
model_RFF.fit(df_encoded , y_train)

In [ ]:
# Affichage de la matrice de confusion
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)

plt.title("Matrice de confusion")
plt.show()

##### Matrrice de confusion avec des classes plus équilibrées


In [ ]:
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN

# Combinaison de sous-échantillonnage et de sur-échantillonnage
tomek = TomekLinks(random_state=42)  # Réduit la majoritaire
adasyn = ADASYN(sampling_strategy='minority', random_state=42)

X_vraitrain, y_vraitrain = adasyn.fit_resample(X_vraitrain, y_vraitrain)
X_vraitrain, y_vraitrain = tomek.fit_resample(X_vraitrain, y_vraitrain)


# Vérification des comptages de classe après sur-échantillonnage
print(f"Nombre d'échantillons par classe avant sur-échantillonnage : {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"Nombre d'échantillons par classe après sur-échantillonnage : {dict(zip(*np.unique(y_vraitrain, return_counts=True)))}")

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=88, n_jobs=-1)
model.fit(X_vraitrain, y_vraitrain)
y_pred = model.predict_proba(X_val)[:, 1]
pr_auc = average_precision_score(y_val, y_pred)
print(f'PR-AUC: {pr_auc}')

from sklearn.metrics import confusion_matrix

# Calculez la matrice de confusion
cm = confusion_matrix(y_val, y_pred)

# Calculez le taux de faux positifs et de faux négatifs
fp_rate = cm[0, 1] / cm[0].sum()
fn_rate = cm[1, 0] / cm[1].sum()

print(f"False positive rate: {fp_rate}")
print(f"False negative rate: {fn_rate}")


#### Soumission des données au site du challenge.

In [ ]:
current_directory = "c:/Users/gabriel/Desktop/Dauphine/Dauphine L2/Semestre 2/MIDO/Intro machine learning réseaux de neurones/projet ml"
train_file_path = os.path.join(current_directory, 'X_test_8skS2ey.csv')

# On accède au fichier test
df_test = pd.read_csv(train_file_path)

# On applique les mêmes transformations que sur le fichier train
df_test = tout(df_test)

In [ ]:
df_test.shape # On affiche la taille de notre fichier test

In [ ]:
df_encoded.shape # On affiche la taille de notre fichier train

In [ ]:
# On vérifie si les colonnes de df_test sont les mêmes que celles de df_encoded
# On sauvegarde la colonne ID de df_test
ID=df_test['ID']
df_test=df_test
df_test = df_test.reindex(columns=df_encoded.columns)
df_test = df_test.fillna(0)

In [ ]:
# On affiche la taille de notre fichier test
df_test.shape 

In [ ]:
# Obtenir les noms des colonnes en tant qu'ensembles
columns_test = set(df_test.columns)
columns_encoded = set(df_encoded.columns)

# Trouver les colonnes qui sont dans df_test mais pas dans df_encoded
diff_test_encoded = columns_test - columns_encoded
print("Colonnes dans df_test mais pas dans df_encoded:", diff_test_encoded)

In [ ]:
# Trouver les colonnes qui sont dans df_encoded mais pas dans df_test
diff_encoded_test = columns_encoded - columns_test
print("Colonnes dans df_encoded mais pas dans df_test:", diff_encoded_test)

In [ ]:
# On prédit les probabilités de fraude et on enregistre les résultats dans un fichier csv
X = model_RFF.predict_proba(df_test)[:, 1]
n = df_test.shape[0]
I = [i for i in range(n)]
csv = pd.DataFrame({'index': I, 'ID': ID, 'fraud_flag': X})
csv.to_csv("c:/Users/gabriel/Desktop/Dauphine/Dauphine L2/Semestre 2/MIDO/Intro machine learning réseaux de neurones/projet ml/submission10", index=False)

In [ ]:
# On affiche la taille de notre ensemble de prédcition
X.shape 

In [ ]:
# On affiche le type de X
type(X) 